[Table of Contents](table_of_contents.ipynb)

# Topic 16.  Recursive Least Squares
Author: Seth Nielsen - sethmnielsen@gmail.com
    

##  Introduction

Least squares approximation is a method used to approximate the solution to a system of equations. Linear least squares can be applied to a set of data to find a model that minimizes the sum of the squared errors between the data and their corresponding modeled values, thus creating a model of best fit. This method can further be applied to create a least-squares filter; a type of filter that, given a sequence of input data, uses least squares to find the coefficients that minimize the error between the filter's output and a desired sequence. 

The traditional form of a least squares filter is performed in *batch* form, where the minimizing solution is computed on an entire block of data after the data has been collected. This method of calculating a solution can be effective for applications suited to offline computation, but is not particularly useful for systems that require real-time or online parameter estimation for parameters that are unknown in advance or are changing. In this case, an adaptive filter is required. The recursive least squares filter modifies the least squares filter to be adaptive by continuously updating the estimated parameters (coefficients) as new data arrive. 

## Explanation of the theory

Give a detailed discussion (i.e., equations galore) of the topic.  
The emphasis here is clarity for future students learning the topic.

Least squares filter:

\begin{equation}
Rh = A^H d, \text{ where } R = A^H A = \sum_{i=1}^N q_{i}\ q_{i}^H \\
\text{with } q_i = \begin{bmatrix}f_{i}\\f_{i-1}\\...\\f_{i-m+1}\end{bmatrix} \\
\end{equation}

\begin{align}
\text{Let}\ \ p &= A^H d = \sum_{i=1}^N q_{i}d_i \\
h &= R^{-1}A^{H} = R^{-1}p \\
\end{align}

Start RLS algorithm:

\begin{equation}
R_t = \sum_{i=1}^t q_{i}q_{i}^H \\
p_t = \sum_{i=1}^t q_{i}d_i = p_{t-1} + q_{t}d_{t} \\
h_t = R_{t}^{-1}p_t \\
\end{equation}

Algorithm is now adaptive, but needs to be recursive ($R_{t}^{-1}$ is currently calculated at each time step). We can break $R_t$ up as

\begin{align}
R_t &= \sum_{i=1}^{t-1}{q_i}{q_{i}^H} + {q_t}{q_{t}^H} \\
    &= R_{t-1} + q_{t}q_{t}^H
\end{align}

By the Sherman-Morrison formula (eq. 4.32),

\begin{equation}
R_{t}^{-1} = R_{t-1}^{-1} - \frac{R_{t-1}^{-1}{q_t}\:{q_{t}^H}\:R_{t-1}^{-1}}{1 + {q_{t}^H}\:{R_{t-1}^{-1}}{q_t}}
\end{equation}

Let

\begin{align}
P_t &= R_{t}^{-1} \\
k_t &= \frac{R_{t-1}^{-1}{q_t}}{1 + {q_{t}^H}{R_{t-1}^{-1}}{q_t}}.
\end{align}

Then we arrive at
\begin{equation}
P_t = P_{t-1} - {k_t}\:{q_{t}^H}{P_{t-1}}. \quad \text{(eq. 4.38)}
\end{equation}

The coefficients for the filter, or the estimated parameters, are computed by

\begin{align}
\hat{h}_t &= P_t p_t = P_t (\ p_{t-1} + q_t\ d_t\ ) \\
&= P_t\ p_{t-1} + P_t\ q_t\ d_t \\
&= P_{t-1}\ p_{t-1} - k_t\ q_t^H\ P_{t-1}\ p_{t-1} + P_t\ q_t\ d_t \ \Leftarrow\ \text{using (eq. 4.38)} \\
&= \hat{h}_{t-1} - k_t\ q_t^H\ \hat{h}_{t-1} + k_t\ d_t \qquad \qquad \quad \Leftarrow\ k_t = P_t\ q_t \\
&= \hat{h}_{t-1} + k_t (\ d_t - q_t^H\ \hat{h}_{t-1}\ ).
\end{align}

The quantity multiplied by $k_t$ respresents the filter error

\begin{equation}
\varepsilon_t = d_t - q_t^H\ \hat{h}_{t-1},
\end{equation}

which allows us to write the update of the filter coefficients in the simple form of

\begin{equation}
\hat{h}_t = \hat{h}_{t-1} + k_t \varepsilon_t.
\end{equation}

### Initialization

To initialize the algorithm, the initial condition $P_0 = R_{0}^{-1}$ is needed. A slight perturbation of some small $\delta$ to the matrix $R_t$ gives the following:
\begin{align}
R_t &= \sum_{i=1}^t q_{i}q_{i}^H + \delta I \\
R_0 &= \delta I \\
P_0 &= \delta^{-1} I
\end{align}

The initial filter coefficients can be assumed to be zero, and thus $\hat{h}_0 = 0$. 

## Simple Numerical Examples

Provide some simple python code and examples that emphasize the basic concepts.

We can start by estimating the parameters of a simple system with impulse response $h = [\ 1,2,3,4,5\ ]$ using a recursive least squares filter. The following filter written in Python will give normally-distributed random values $f_t$ as inputs and attempt to match the system's true output $d_t$ by computing the error $\varepsilon_t = d_t - y_t$ , where $y_t$ is the filtered output evaluated as $y_t = q_{t}^H \ \hat{h}_t$, and $\hat{h}_t$ is the vector of estimated parameters that is updated by computing $\hat{h}_t = \hat{h}_{t-1} + k_t \varepsilon_t$.

In [6]:
import numpy as np
np.set_printoptions(precision=4)


m = 5      # number of parameters
n = 1000   # iterations

h = np.array([1,2,3,4,5])  # impulse response
hhat = np.zeros(m)         # initial estimated parameters
f = np.random.randn(n)     # normally-distributed random input
fn = np.hstack(([0,0,0,0],f))  # convenience array used to shift through input (f)
q = np.zeros(m)  # input data for one time step
delta = .0001
P = 1/delta * np.eye(m)  # initialize P[0] without calculating inverse of R[0]


for i in range(n):
    d = fn[i:i+5] @ h  # true output
    q = fn[i:i+m]      # q update

    k = P @ q / (1 + q.T @ P @ q)  # kalman gain vector
    y = q @ hhat  # filter output
    e = d - y     # error
    
    hhat = hhat + k * e  # update of estimated parameters
    P = P - k * q @ P    # P update

print('hhat:',hhat)

hhat: [1. 2. 3. 4. 5.]


## An Engineering Application

Provide a more sophisticated example showing one engineering example of the topic, complete with python code.

Mass-Spring-Damper